In [15]:
from pathlib import Path
import cv2

sift = cv2.SIFT_create(
    nOctaveLayers=7,
    sigma=1.414
)


def load_logos(base_folder: str):
    logos_dict = {}
    base_path = Path(base_folder)

    for brand_folder in base_path.iterdir():
        if brand_folder.is_dir():
            brand_name = brand_folder.name
            logos = [str(file) for file in brand_folder.glob("*") if file.is_file()]
            logos_dict[brand_name] = logos

    return logos_dict

folder = "Logos" 
logos_paths = load_logos(folder)

print(logos_paths)
descriptores = {}


for brand, paths in logos_paths.items():
    descriptores[brand] = []
    for path in paths:
        logo = cv2.imread(path)
        
        if logo is None:
            print(f"Error loading image: {path}")
            continue

        gray_logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
        kp_logo, des_logo = sift.detectAndCompute(logo, None)
        descriptores[brand].append((kp_logo, des_logo))



{'Adidas': ['Logos\\Adidas\\logo1.png'], 'Nike': ['Logos\\Nike\\logo1.png', 'Logos\\Nike\\logo10.png', 'Logos\\Nike\\logo11.png', 'Logos\\Nike\\logo12.png', 'Logos\\Nike\\logo13.jpg', 'Logos\\Nike\\logo14.png', 'Logos\\Nike\\logo15.png', 'Logos\\Nike\\logo16.png', 'Logos\\Nike\\logo17.png', 'Logos\\Nike\\logo18.png', 'Logos\\Nike\\logo19.png', 'Logos\\Nike\\logo2.png', 'Logos\\Nike\\logo20.png', 'Logos\\Nike\\logo3.png', 'Logos\\Nike\\logo4.png', 'Logos\\Nike\\logo5.png', 'Logos\\Nike\\logo6.png', 'Logos\\Nike\\logo7.png', 'Logos\\Nike\\logo9.png'], 'Puma': ['Logos\\Puma\\logo1.png']}


In [ ]:
import cv2
import numpy as np
from pathlib import Path

# Load all cloth images from the Cloth folder
cloth_folder = Path("Cloth")
cloth_images = list(cloth_folder.glob("*"))

# Process each cloth image
for cloth_path in cloth_images:
    if not cloth_path.is_file():
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {cloth_path.name}")
    print('='*60)
    
    shirt = cv2.imread(str(cloth_path), cv2.IMREAD_GRAYSCALE)
    
    if shirt is None:
        print(f"Error loading image: {cloth_path}")
        continue
    
    # Compute SIFT features directly on the shirt image
    kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)
    
    if des_shirt is None:
        print("No descriptors found in shirt")
        continue
    
    print(f"Found {len(kp_shirt)} SIFT keypoints")
    
    best_brand = None
    best_match_count = 0
    best_matches = None
    best_logo_kp = None
    best_H = None
    best_mask = None
    best_logo_index = None
    
    # Match against all logos from all brands
    for brand, logo_data in descriptores.items():
        for idx, (kp_logo, des_logo) in enumerate(logo_data):
            if des_logo is None:
                continue
            
            # Match with KNN
            bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
            knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)
            
            # Lowe's ratio test
            good = []
            ratio_thresh = 0.6
            
            for match_pair in knn_matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_thresh * n.distance:
                        good.append(m)
            
            # Check if this is the best match so far
            if len(good) > best_match_count:
                best_match_count = len(good)
                best_brand = brand
                best_matches = good
                best_logo_kp = kp_logo
                best_logo_index = idx
                
                # Compute homography if enough matches
                MIN_MATCH_COUNT = 4
                if len(good) >= MIN_MATCH_COUNT:
                    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                    best_H = H
                    best_mask = mask
    
    # Display results for this cloth
    print(f"Best match: {best_brand} with {best_match_count} good matches")
    
    if best_matches and best_match_count >= 4:
        # Load the best matching logo for visualization
        logo_path = logos_paths[best_brand][best_logo_index]
        logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
        
        matchesMask = best_mask.ravel().tolist() if best_mask is not None else None
        
        # Draw matches
        result = cv2.drawMatches(
            logo, best_logo_kp,
            shirt, kp_shirt,
            best_matches, None,
            matchesMask=matchesMask,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        cv2.namedWindow(f"Matches - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Matches - {cloth_path.name}", result)
        cv2.resizeWindow(f"Matches - {cloth_path.name}", 1000, 1000)
        
        # Draw bounding box on original shirt image
        if best_H is not None:
            h, w = logo.shape
            logo_corners = np.float32([
                [0, 0],
                [0, h - 1],
                [w - 1, h - 1],
                [w - 1, 0]
            ]).reshape(-1, 1, 2)
            
            projected_corners = cv2.perspectiveTransform(logo_corners, best_H)
            
            shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
            shirt_with_box = cv2.polylines(
                shirt_color,
                [np.int32(projected_corners)],
                isClosed=True,
                color=(0, 255, 0),
                thickness=3,
                lineType=cv2.LINE_AA
            )
            
            cv2.namedWindow(f"Detection - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"Detection - {cloth_path.name}", shirt_with_box)
            cv2.resizeWindow(f"Detection - {cloth_path.name}", 800, 800)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print(f"Not enough matches found for detection")


Processing: nike_shirt1.png
Found 1567 SIFT keypoints
Best match: Nike with 53 good matches

Processing: nike_shirt2.png
Found 9709 SIFT keypoints
Best match: Nike with 120 good matches

Processing: nike_shirt3.png
Found 1608 SIFT keypoints
Best match: Nike with 38 good matches

Processing: nike_shirt5.png
Found 1957 SIFT keypoints
Best match: Nike with 9 good matches

Processing: nike_shirt6.png
Found 3060 SIFT keypoints
Best match: Nike with 38 good matches

Processing: nike_shoe1.png
Found 4704 SIFT keypoints
Best match: Nike with 13 good matches

Processing: nike_shoe2.png
Found 5182 SIFT keypoints
Best match: Nike with 15 good matches
